In [1]:
import os
import fiona
import hashlib
import boto3
import json
import datacube
from datacube.api import GridWorkflow
from madmex.wrappers import gwf_query
from madmex.models import SegmentationInformation
from madmex.util import parser_extra_args
from madmex.settings import TEMP_DIR
from importlib import import_module
import gc

In [2]:
name_of_product = 's2_10m_fiona_test_2_2018'
gwf_kwargs = {'region': 'Jalisco', 'begin': '2018-01-01', 'end':'2018-12-31'}
gwf_kwargs.update(product=name_of_product)

In [3]:
iterable = gwf_query(**gwf_kwargs)

In [4]:
lista_iter = list(iterable)

In [13]:
[lista_iter[x][0] for x in range(0,len(lista_iter))]

[(28, -27),
 (28, -28),
 (30, -29),
 (30, -30),
 (28, -29),
 (28, -30),
 (29, -30),
 (30, -27),
 (29, -29),
 (30, -28),
 (29, -28),
 (29, -27)]

In [14]:
var = lista_iter[8]

In [25]:
algorithm = 'bis' #'slic'
datasource = 'sentinel2'
name = 's2_10m_some_scenes_fiona_test'
year = '2018'
extra_args = parser_extra_args(['t=30','s=0.5','c=0.7']) #for slic: parser_extra_args(['n_segments=10000', 'compactness=0.5'])

In [16]:
dataset_name = datasource + '_%d_%d_' % (var[0][0], var[0][1]) + year

In [17]:
dataset_name

'sentinel2_29_-29_2018'

In [10]:
path = os.path.join(TEMP_DIR, 'segmentation_results')

In [11]:
if not os.path.exists(path):
    os.makedirs(path)

In [22]:
band_list = ['ndvi_mean']

In [20]:
try:
    module = import_module('madmex.segmentation.%s' % algorithm)
    Segmentation = module.Segmentation
except ImportError as e:
    raise ValueError('Invalid model argument')

In [23]:
geoarray = GridWorkflow.load(var[1], measurements=band_list)

In [26]:
seg = Segmentation.from_geoarray(geoarray, **extra_args)

In [27]:
seg.crs

'+proj=lcc +lat_1=17.5 +lat_2=29.5 +lat_0=12 +lon_0=-102 +x_0=2500000 +y_0=0 +a=6378137 +b=6378136.027241431 +units=m +no_defs '

In [29]:
var[1].geobox

GeoBox(5002, 5002, Affine(10.0, 0.0, 2427740.0,
       0.0, -10.0, 1026160.0), PROJCS["unnamed",GEOGCS["WGS 84",DATUM["unknown",SPHEROID["WGS84",6378137,6556752.3141]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["standard_parallel_1",17.5],PARAMETER["standard_parallel_2",29.5],PARAMETER["latitude_of_origin",12],PARAMETER["central_meridian",-102],PARAMETER["false_easting",2500000],PARAMETER["false_northing",0]])

In [30]:
var[1].geobox.extent

Geometry(POLYGON ((2427740 1026160,2427740 976140,2477760 976140,2477760 1026160,2427740 1026160)), PROJCS["unnamed",GEOGCS["WGS 84",DATUM["unknown",SPHEROID["WGS84",6378137,6556752.3141]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["standard_parallel_1",17.5],PARAMETER["standard_parallel_2",29.5],PARAMETER["latitude_of_origin",12],PARAMETER["central_meridian",-102],PARAMETER["false_easting",2500000],PARAMETER["false_northing",0]])

In [31]:
var[1].geobox.extent.wkt

'POLYGON ((2427740 1026160,2427740 976140,2477760 976140,2477760 1026160,2427740 1026160))'

In [34]:
var[1].geobox.extent.json

{'type': 'Polygon',
 'coordinates': [[(2427740.0, 1026160.0),
   (2427740.0, 976140.0),
   (2477760.0, 976140.0),
   (2477760.0, 1026160.0),
   (2427740.0, 1026160.0)]]}

In [14]:
try:
    seg.segment()
    # Try deallocating input array
    seg.array = None
    geoarray = None
    seg.polygonize(crs_out=None)  #None for no reprojection to latlon
    # Write to shapefile
    hash = hashlib.md5(dataset_name.encode('utf-8')).hexdigest()[0:6]
    name_file = hash + '_' + dataset_name
    out_file = path + '/' + name_file
    schema = {'geometry': 'Polygon',
          'properties': [('id', 'int')]}
    with fiona.open(path, 'w', layer = name_file,
                    schema=schema,
                    driver='ESRI Shapefile',
                    crs=seg.crs) as dst:
        for feature in seg.fc:
            dst.write(feature)

    #Upload to s3 bucket
    s3 = boto3.client('s3')
    bucket_name = 'my-segmentation-bucket'
    filename = name_file + '.shp'
    filepath = path + '/' + filename
    s3.upload_file(filepath, bucket_name, filename)
    os.remove(filepath)
    filename = name_file + '.shx'
    filepath = path + '/' + filename
    s3.upload_file(filepath, bucket_name, filename)
    os.remove(filepath)
    filename = name_file + '.cpg'
    filepath = path + '/' + filename
    s3.upload_file(filepath, bucket_name, filename)
    os.remove(filepath)
    filename = name_file + '.dbf'
    filepath = path + '/' + filename
    s3.upload_file(filepath, bucket_name, filename)
    os.remove(filepath)
    filename = name_file + '.prj'
    filepath = path + '/' + filename
    s3.upload_file(filepath, bucket_name, filename)
    os.remove(filepath)
    gc.collect()
except Exception as e:
        print(e)

  Anticipated allocation: 879609KB
    ...allocated
  thresh 1
  thresh 2
  thresh 3
  thresh 4
  thresh 5
  thresh 6
  thresh 7
  thresh 8
  thresh 9
  thresh 10
  thresh 11
  thresh 12
  thresh 13
  thresh 14
  thresh 15
  thresh 16
  thresh 17
  thresh 18
  thresh 19
  thresh 20
  thresh 21
  thresh 22
  thresh 23
  thresh 24
  thresh 25
  thresh 26
  thresh 27
  thresh 28
  thresh 29
  thresh 30
